In [31]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [32]:
%hive_start

In [33]:
%timeout 300

In [34]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [35]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

Deleted /tmp/output
copyFromLocal: `data.tsv': No such file or directory
ls: `/tmp/output/*': No such file or directory


In [36]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;



--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 11.548 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.407 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.113 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.015 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.15 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 0.929 seconds


In [37]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT
        tbl0.c1,
        tbl0.c2,
        tbl1.c4[tbl0.c2]
    FROM
        tbl0
    JOIN (
        SELECT
            c1,
            c4
        FROM
            tbl1
        ) tbl1
    ON
        (tbl0.c1 = tbl1.c1);

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 0.777 seconds
CREATE TABLE salida
AS
    SELECT
        tbl0.c1,
        tbl0.c2,
        tbl1.c4[tbl0.c2]
    FROM
        tbl0
    JOIN (
        SELECT
            c1,
            c4
        FROM
            tbl1
        ) tbl1
    ON
        (tbl0.c1 = tbl1.c1);
Query ID = root_20200120014026_549adb1f-55e8-43c9-81df-588a1ff10aff
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-01-20 01:40:38	Starting to launch local task to process map jo

In [38]:
%%hive
DROP TABLE IF EXISTS tbl1;
DROP TABLE IF EXISTS tbl0;

DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.218 seconds
DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.131 seconds


In [39]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120014123_e6c4acac-1dfb-4acd-9167-6e868f4d6604
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579483283791_0008, Tracking URL = http://8ac6a4831d11:8088/proxy/application_1579483283791_0008/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579483283791_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 01:41:32,856 Stage-1 map = 0%,  reduce = 0%
2020-01-20 01:41:39,644 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.21 sec
MapReduce Total cumulative CPU time: 2 seconds 210 msec
Ended Job = job_1579483283791_0008
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000

In [40]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [ ]:
%hive_quit

In [4]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

Deleted /tmp/output
copyFromLocal: `data.tsv': No such file or directory
ls: `/tmp/output/*': No such file or directory


In [5]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 22.773 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.598 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.595 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.699 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.187 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 2.574 seconds


In [6]:
%%hive
SELECT
    concat(tbl0.c1,',',
    tbl0.c2,',',
    tbl1.c4[tbl0.c2])
FROM
    tbl0
JOIN (
    SELECT
        c1,
        c4
    FROM
        tbl1
    ) tbl1
ON
    (tbl0.c1 = tbl1.c1);


SELECT
    tbl0.c1 AS numer,
    tbl0.c2 AS letra,
    tbl1.c4[tbl0.c2] AS valor
FROM
    tbl0
JOIN (
    SELECT
        c1,
        c4
    FROM
        tbl1
    ) tbl1
ON
    (tbl0.c1 = tbl1.c1);
Query ID = root_20200116043257_79135ef6-06d1-4fdd-b1fe-0bc74ac9c68d
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-01-16 04:33:23	Starting to launch local task to process map join;	maximum memory = 477626368
2020-01-16 04:33:27	Dump the side-table for tag: 1 with group count: 10 into file: file:/tmp/root/9bf1df7a-ca38-49ae-b1cf-87553dcbb35c/hive_2020

In [7]:
%%hive
DROP TABLE IF EXISTS tbl1;
DROP TABLE IF EXISTS tbl0;

DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.225 seconds
DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.24 seconds


In [8]:
%hive_quit